# 보안, 금융, 위험 관리, 법률, 재해 복구, IT 거버넌스

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U langchain-community
!pip install transformers
!pip install datasets
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 5.4 MB/s eta 0:00:00


In [ ]:
import re
import json
import torch
import pandas as pd
import re
import tqdm

from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from langchain.document_loaders import PyPDFLoader
from statistics import mean
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

import warnings
warnings.filterwarnings("ignore")

# QA 데이터 생성

In [ ]:
# model_name = "yanolja/EEVE-Korean-10.8B-v1.0"
# model_name = 'beomi/KoAlpaca-Polyglot-12.8B'
# model_name = 'upstage/SOLAR-10.7B-Instruct-v1.0'
model_name = 'skt/A.X-4.0-Light'

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          cache_dir='/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/qaset_tokenizer',
                                          padding_side = "left")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype = torch.float16,
    device_map = "auto",
    cache_dir = '/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/qaset_model'
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/130 [00:00<?, ?B/s]

In [ ]:
with open('/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/all_chunks.json', "r", encoding = "utf-8") as f:
  all_chunks = json.load(f)
  all_chunks = [
      {'page_content': chunk['page_content']}
      for chunk in all_chunks
  ]

In [ ]:
# contexts = [doc['page_content'] for doc in all_chunks]
contexts = all_chunks

all_prompts = []

for context in contexts:
    all_prompts.append(
        f"""
        아래 문단을 바탕으로 객관식 질문 1개를 만들어줘.
        단, 객관식은 1 / 2 / 3 / 4 / 5 형태로 5지선다로 하고 정답 표시 포함.
        답안은 1 / 2 / 3 / 4 / 5 같이 숫자만 표시.
        아래의 문단을 바탕으로 주관식 질문 1개를 만들어줘.
        단, 주관식은 질문에 대한 답안 포함.

        문단 :
        {context}
        """
    )

In [ ]:
# 배치 처리
batch_size = 32
qa_dataset = []

for i in tqdm.tqdm(range(0, len(all_prompts), batch_size), desc = "QA Dataset..."):
    batch_prompts = all_prompts[i:i+batch_size]
    inputs = tokenizer(batch_prompts, return_tensors = "pt", padding = True, truncation = False).to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens = 2000,
            do_sample = False,
            eos_token_id = tokenizer.eos_token_id
        )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens = True)
    for context, qa_text in zip(contexts[i:i+batch_size], decoded):
        qa_dataset.append({"context": context, "qa": qa_text})

print(f"총 {len(qa_dataset)}개의 QA 생성 완료")

QA Dataset...: 100%|██████████| 27/27 [09:15<00:00, 20.57s/it]

총 851개의 QA 생성 완료


In [ ]:
# qa_dataset 저장
with open("/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/qa_dataset7.json", "w", encoding = "utf-8") as f:
    for item in qa_dataset:
        json.dump({
            "instruction" : "다음 문단에 대한 질문에 답하시오.",
            "input" : item["context"],
            "output" : item["qa"]
        }, f, ensure_ascii = False)
        f.write("\n")

# QA 데이터셋 합치기

In [ ]:
import json
import glob

merged = []

for file in glob.glob("/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/qa/*.json"):
    with open(file, "r", encoding="utf-8") as f:
        try:
            # JSON Array
            data = json.load(f)
            if isinstance(data, dict):  # 단일 객체면 리스트로
                data = [data]
        except json.JSONDecodeError:
            # JSON Lines
            f.seek(0)
            data = [json.loads(line) for line in f] # 한줄씩 읽어서 리스트로

        for item in data:
            if "instruction" in item:
                merged.append({
                    "instruction": item.get("instruction", ""),
                    "input": item.get("input", ""),
                    "output": item.get("output", "")
                })
            elif "page_content" in item:
                merged.append({
                    "instruction": "다음 내용을 기반으로 답변하세요.",
                    "input": item.get("page_content", ""),
                    "output": ""
                })

In [ ]:
# 특수문자 제거
pattern = r'[①-⑳◆·「」｢｣□◦\-\*❶-❸Ⅲ※‧•Ÿ]'
for item in merged:
  text = item.get("input")

  if isinstance(text, dict):
    text = text['page_content'] # Access the string content

  if isinstance(text, str):
    text = re.sub(pattern, ' ', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    item["input"] = text.strip()

  output_text = item.get('output')
  if isinstance(output_text, dict):
    output_text = output_text['page_content'] # Access the string content

  if isinstance(output_text, str):
    output_text = re.sub(pattern, ' ', output_text)
    output_text = re.sub(r'\n', ' ', output_text)
    output_text = re.sub(r'\s+', ' ', output_text)
    item["output"] = output_text.strip()

In [ ]:
with open("/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/merged_qa.json", "w", encoding="utf-8") as f:
    json.dump(merged, f, ensure_ascii=False, indent=2)

print(len(merged))

11759
